In [5]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta
import sqlite3
import csv

In [6]:
table = pd.read_csv('/Users/michaelmitros/Desktop/Sheet 1-1-OpenTable.csv')
# Make sure column names are strings first
table.columns = table.columns.astype(str)
# Replace spaces with underscores
table.columns = table.columns.str.replace(' ', '_')
table.head()

/var/folders/rc/pb3j0f_913z1_y_9mwnxgbph0000gn/T/ipykernel_24844/3240501319.py:1: DtypeWarning: Columns (5,6,14,15,20,22,26,27,28,29,34) have mixed types. Specify dtype option on import or set low_memory=False.
  table = pd.read_csv('/Users/michaelmitros/Desktop/Sheet 1-1-OpenTable.csv')


,Visit_Date,Visit_Time,Created_Date,Created_Time,Created_Lead_Time,Canceled_Date,No-Show_Date,Seated_Date,Finished_Date,Total_Actual_Duration,...,Experience_Title,Experience_Price_Type,Experience_Price,Experience_Total_Revenue,Experience_Gratuity,Reservation_Tags,First_Visit_Date,Completed_Visits,Canceled_Count,No-Show_Count
0,2025-06-05,07:30 PM,2025-06-02,10:31 PM,4138,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2022-05-13 20:00,3.0,3.0,0.0
1,2025-06-05,07:30 PM,2025-06-02,02:04 PM,4645,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0
2,2025-06-05,07:15 PM,2025-06-04,02:06 PM,1748,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Anniversary,NaN,0.0,0.0,0.0
3,2025-06-05,07:00 PM,2025-06-04,09:23 AM,2016,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2019-06-01 17:30,2.0,0.0,0.0
4,2025-06-05,06:45 PM,2025-05-27,12:32 PM,13332,2025-06-05 11:55,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0


In [7]:
#Convert date columns
date_cols = [
    'Visit_Date', 'Created_Date', 'Canceled_Date', 'No-Show_Date',
    'Seated_Date', 'Finished_Date', 'First_Visit_Date'
]

for col in date_cols:
    table[col] = pd.to_datetime(table[col], errors='coerce')
# Convert number columns
num_cols = ['Experience_Price', 'Completed_Visits', 'Canceled_Count', 'No-Show_Count']
for col in num_cols:
    table[col] = pd.to_numeric(table[col], errors='coerce')

In [8]:
#Drop columns with >90% missing
table = table.loc[:, table.isnull().mean() < 0.9].copy()
#Fill missing
table.loc[:, 'Online_Channel'] = table['Online_Channel'].fillna('None')
table.loc[:, 'Table'] = table['Table'].fillna('Unknown')
#Delete dupes
table.drop_duplicates(inplace=True)

In [9]:
#Check missing values by column
missing_report = table.isnull().mean().sort_values(ascending=False)
print(missing_report[missing_report > 0])

Table_Type               0.897451
Dining_Area_Assigned     0.890572
Reservation_Tags         0.864764
Canceled_Date            0.849125
Partner                  0.776914
Server                   0.702309
Total_Actual_Duration    0.382037
Finished_Date            0.379195
Seated_Date              0.319119
First_Visit_Date         0.142650
Completed_Visits         0.015468
Canceled_Count           0.015446
No-Show_Count            0.015446
dtype: float64


In [10]:
# Adds Season Column
table['date'] = table['Visit_Date'].dt.date
table['weekday'] = table['Visit_Date'].dt.day_name()
table['hour'] = table['Visit_Date'].dt.hour
table['month'] = table['Visit_Date'].dt.month
table['season'] = table['month'].map({
    12: 'Winter', 1: 'Winter', 2: 'Winter',
    3: 'Spring', 4: 'Spring', 5: 'Spring',
    6: 'Summer', 7: 'Summer', 8: 'Summer',
    9: 'Fall', 10: 'Fall', 11: 'Fall'
})

In [14]:
#Filter out closed days
closed_days = ['Sunday', 'Monday', 'Tuesday']
table = table[~table['weekday'].isin(closed_days)]
#Convert to 24hour
table['Visit_Time_24'] = pd.to_datetime(table['Visit_Time'], format='%I:%M %p').dt.strftime('%H:%M')


In [15]:
# Now filter with allowed_times on this new column
allowed_times = [
    "16:00", "16:15", "16:30", "16:45",
    "17:00", "17:15", "17:30", "17:45",
    "18:00", "18:15", "18:30", "18:45",
    "19:00", "19:15", "19:30", "19:45",
    "20:00", "20:15", "20:30", "20:45",
    "21:00", "21:15", "21:30"
]
filtered_table = table[
    (~table['weekday'].isin(['Sunday', 'Monday', 'Tuesday'])) &  # example closed days
    (table['Visit_Time_24'].isin(allowed_times))
]

In [17]:

volume_by_slot = (
    filtered_table
    .groupby(['season', 'weekday', 'Visit_Time_24'])
    .size()
    .reset_index(name='reservations')
)

In [23]:

table.to_csv('OpenTable(Everything).csv', index=False)
print("Exported to OpenTable(Everything).csv")

Exported to OpenTable(Everything).csv


In [18]:
# Build full date time slot grid
start_date = filtered_table['Visit_Date'].min().normalize()
end_date = filtered_table['Visit_Date'].max().normalize()
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

valid_days = ['Wednesday', 'Thursday', 'Friday', 'Saturday']
valid_dates = [d for d in date_range if d.day_name() in valid_days]

time_slots = pd.date_range("16:00", "21:30", freq="15min").time

# Cross join dates and time slots
full_slots = pd.DataFrame(
    [(date, t.strftime("%H:%M")) for date in valid_dates for t in time_slots],
    columns=["Visit_Date", "Visit_Time_24"]
)

# Normalize Visit_Date in filtered_table
filtered_table.loc[:, 'Visit_Date'] = filtered_table['Visit_Date'].dt.normalize()




In [19]:
#Aggregate guest counts per slot
reservation_counts = (
    filtered_table.groupby(['Visit_Date', 'Visit_Time_24'])['Size']
    .sum()
    .reset_index()
    .rename(columns={'Size': 'num_guests_seated'})
)

# Merge full slot grid with aggregated counts
all_slots = pd.merge(
    full_slots,
    reservation_counts,
    how='left',
    on=['Visit_Date', 'Visit_Time_24']
)

# Fill missing guest counts with 0
all_slots['num_guests_seated'] = all_slots['num_guests_seated'].fillna(0).astype(int)

# Add boolean flag for slot usage
all_slots['slot_was_used'] = all_slots['num_guests_seated'] > 0

In [21]:

# Add weekday, month, and season columns
all_slots['weekday'] = all_slots['Visit_Date'].dt.day_name()
all_slots['month'] = all_slots['Visit_Date'].dt.month
all_slots['season'] = all_slots['month'].map({
    12: 'Winter', 1: 'Winter', 2: 'Winter',
    3: 'Spring', 4: 'Spring', 5: 'Spring',
    6: 'Summer', 7: 'Summer', 8: 'Summer',
    9: 'Fall', 10: 'Fall', 11: 'Fall'
})



 Exported to reservation_slot_counts_detailed2.csv


In [22]:
# Export final grid with counts
all_slots.to_csv('OpenTable2.csv', index=False)
print(" Exported to OpenTable2.csv")

,Visit_Date,Visit_Time_24,num_guests_seated,slot_was_used,weekday,month,season
0,2018-06-22,16:00,0,False,Friday,6,Summer
1,2018-06-22,16:15,0,False,Friday,6,Summer
2,2018-06-22,16:30,0,False,Friday,6,Summer
3,2018-06-22,16:45,0,False,Friday,6,Summer
4,2018-06-22,17:00,0,False,Friday,6,Summer
5,2018-06-22,17:15,0,False,Friday,6,Summer
6,2018-06-22,17:30,0,False,Friday,6,Summer
7,2018-06-22,17:45,0,False,Friday,6,Summer
8,2018-06-22,18:00,0,False,Friday,6,Summer
9,2018-06-22,18:15,0,False,Friday,6,Summer
